# 00 — Setup y verificación de GPU

Este notebook verifica el entorno de ejecución (versiones, GPU, seeds y rutas) y deja configuradas variables comunes que usarán los siguientes notebooks del experimento **E0 (ablación de covariables)**.

## Notebooks (orden de ejecución)
1. **00_setup_and_gpu_check.ipynb** (este) — entorno, GPU, seeds, paths.
2. **01_Setup_and_Data_Audit.ipynb** — validación, EDA mínima, split temporal fijo, metadata.
3. **02_data_and_feature_sets.ipynb** — inspección del dataset y definición formal de FS0/FS1/FS2 con checks anti-leakage.
4. **08_run_E0_ablation_training.ipynb** — entrenamiento + evaluación end-to-end (GPU) para FS0/FS1/FS2.
5. **09_results_summary_and_plots.ipynb** — consolidación de métricas, deltas y visualizaciones.

## Reproducibilidad (qué se guardará)
- Artefactos del experimento E0 en `outputs/E0_ablation/` (métricas, predicciones si aplica, runtime, device).
- Metadatos mínimos: seed, configuración, versiones de librerías.

**Nota**: no se inventan resultados. Si falta información, se usará `[COMPLETAR: ...]`.

In [1]:
# --- Imports base (safe) ---
from __future__ import annotations

import os
import sys
from pathlib import Path


# Ensure PROJECT_ROOT is on sys.path so `import src.*` works reliably
NOTEBOOK_DIR = Path.cwd()
PROJECT_ROOT = NOTEBOOK_DIR
# If we are inside notebooks/, go up one level
if (PROJECT_ROOT / 'src').exists() is False and PROJECT_ROOT.parent.exists():
    if (PROJECT_ROOT.parent / 'src').exists():
        PROJECT_ROOT = PROJECT_ROOT.parent

sys.path.insert(0, str(PROJECT_ROOT))

print('NOTEBOOK_DIR:', NOTEBOOK_DIR)
print('PROJECT_ROOT:', PROJECT_ROOT)
print('sys.path[0]:', sys.path[0])

NOTEBOOK_DIR: /home/sagemaker-user/TFMAXEL/notebooks
PROJECT_ROOT: /home/sagemaker-user/TFMAXEL
sys.path[0]: /home/sagemaker-user/TFMAXEL


In [2]:
# --- Versiones del entorno ---
from src.e0_ablation_utils import collect_versions

versions = collect_versions()
for k in sorted(versions.keys()):
    print(f'{k}: {versions[k]}')

numpy: 1.24.4
pandas: 2.0.3
platform: Linux-5.10.245-245.983.amzn2.x86_64-x86_64-with-glibc2.35
prophet: 1.1.5
python: 3.11.11
scikit_learn: 1.2.2
statsmodels: 0.14.1
torch: 2.5.1+cu124
torch_cuda: 12.4


In [3]:
# --- Check GPU / device ---

from src.e0_ablation_utils import get_torch_device



device, device_details = get_torch_device(prefer_cuda=True)

print('Selected device:', device)

print('Details:', device_details)



# Optional: sanity check that a tiny tensor can move to GPU

try:

    import torch

    x = torch.randn(4, 4)

    x = x.to(device)

    print('Tensor device OK:', x.device)



    if device == 'cpu':

        print(

            'NOTE: CUDA not available (CPU-only run). '

            'If you expected GPU, install a CUDA-enabled PyTorch build '

            'and ensure NVIDIA drivers are installed.'

        )

except Exception as exc:

    print('Torch/GPU quick check failed:', exc)


Selected device: cuda
Details: {'torch_cuda_available': True, 'cuda_device_name': 'Tesla T4', 'cuda_device_count': 1, 'cuda_capability': (7, 5)}
Tensor device OK: cuda:0


## Seeds y determinismo
En GPU, activar determinismo completo puede **reducir rendimiento** y no siempre está disponible para todas las operaciones.

- Por defecto fijamos seed para `random`, `numpy`, `torch`.
- Si necesitas reproducibilidad bit-a-bit, activa `deterministic=True` (trade-off: más lento).

In [4]:
# --- Seeds ---
from src.e0_ablation_utils import set_global_seed

SEED = 42
DETERMINISTIC = False  # [COMPLETAR: decide si necesitas determinismo estricto]

seed_info = set_global_seed(SEED, deterministic=DETERMINISTIC)
print(seed_info)

{'seed': 42, 'deterministic': False, 'numpy': 'ok', 'torch': 'ok'}


## Paths
Definimos rutas estándar para que el experimento E0 sea reproducible y deje los artefactos en una ubicación fija.

- `DATA_PATH`: CSV de Walmart (se busca en `data/Walmart_Sales.csv` o `Walmart_Sales.csv`).
- `OUTPUT_DIR`: `outputs/E0_ablation/`

In [5]:
from src.e0_ablation_utils import get_project_paths

paths = get_project_paths(project_root=PROJECT_ROOT, output_dir='outputs/E0_ablation')
PROJECT_ROOT = paths.project_root
DATA_PATH = paths.data_path
OUTPUT_DIR = paths.output_dir

print('PROJECT_ROOT:', PROJECT_ROOT)
print('DATA_PATH:', DATA_PATH)
print('OUTPUT_DIR:', OUTPUT_DIR)
print('DATA_PATH exists:', DATA_PATH.exists())

PROJECT_ROOT: /home/sagemaker-user/TFMAXEL
DATA_PATH: /home/sagemaker-user/TFMAXEL/data/Walmart_Sales.csv
OUTPUT_DIR: /home/sagemaker-user/TFMAXEL/outputs/E0_ablation
DATA_PATH exists: True


In [6]:
# --- Quick load sanity check (uses repo loader) ---
# This ensures the environment and paths are correct before moving on.
from src.common import load_data

df = load_data(DATA_PATH)
print('df shape:', df.shape)
print('columns:', list(df.columns))
display(df.head())

df shape: (6435, 8)
columns: ['Store', 'Date', 'Weekly_Sales', 'Holiday_Flag', 'Temperature', 'Fuel_Price', 'CPI', 'Unemployment']


,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment
0,1,2010-02-05,1643690.90,0,42.31,2.572,211.096358,8.106
1,1,2010-02-12,1641957.44,1,38.51,2.548,211.242170,8.106
2,1,2010-02-19,1611968.17,0,39.93,2.514,211.289143,8.106
3,1,2010-02-26,1409727.59,0,46.63,2.561,211.319643,8.106
4,1,2010-03-05,1554806.68,0,46.50,2.625,211.350143,8.106


## Checklist (antes de seguir)
- Si `Selected device` es `cuda`, el entrenamiento podrá usar GPU (en el notebook 08).
- Si `DATA_PATH exists` es `False`, corrige la ruta o copia el CSV a `data/Walmart_Sales.csv`.

Siguiente: ejecutar el notebook **02_data_and_feature_sets.ipynb**.